In [4]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
import numpy as np
import os
import random
from PIL import Image
from math import sqrt
import csv

# -------- CONFIGURATION FLAGS --------
USE_AUGMENTATION = False  # 🔁 Set this to True to enable data augmentation

# -------- PARAMETERS --------
dataset_path = "dataset"
img_size = 128
batch_size = 32
epochs = 10
learning_rate = 0.001
seed = 42
num_classes = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_names = ['dry', 'normal', 'oily']

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# -------- TRANSFORMS --------
normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])

base_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    normalize
])

augment_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    normalize
])

# -------- LOAD DATA --------
full_dataset = datasets.ImageFolder(root=dataset_path, transform=base_transform)
augmented_dataset = datasets.ImageFolder(root=dataset_path, transform=augment_transform)

total_len = len(full_dataset)
train_len = int(0.8 * total_len)
test_len = total_len - train_len

train_base, test_base = random_split(full_dataset, [train_len, test_len])
train_aug, _ = random_split(augmented_dataset, [train_len, test_len])

# -------- DATALOADERS --------
def get_loaders(use_augmented):
    train_data = train_aug if use_augmented else train_base
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_base, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

# -------- MODEL --------
def get_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = True
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, num_classes)
    )
    return model.to(device)

# -------- METRICS --------
def calculate_metrics(y_true, y_pred, average='macro'):
    precision = precision_score(y_true, y_pred, average=average, zero_division=0)
    recall = recall_score(y_true, y_pred, average=average, zero_division=0)
    f1 = f1_score(y_true, y_pred, average=average, zero_division=0)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    try:
        roc_auc = roc_auc_score(torch.nn.functional.one_hot(torch.tensor(y_true), num_classes=num_classes),
                                torch.nn.functional.one_hot(torch.tensor(y_pred), num_classes=num_classes),
                                average=average, multi_class='ovo')
    except:
        roc_auc = None
    return precision, recall, f1, rmse, roc_auc

# -------- TRAINING --------
def train_model(use_augmented):
    print(f"\n{'='*10} Training with {'Augmentation' if use_augmented else 'No Augmentation'} {'='*10}")
    train_loader, val_loader = get_loaders(use_augmented)
    model = get_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    best_val_acc = 0
    best_model_wts = None

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        train_preds, train_labels = [], []

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_acc = train_correct / train_total
        train_precision, train_recall, train_f1, train_rmse, train_roc_auc = calculate_metrics(train_labels, train_preds)

        model.eval()
        val_correct, val_total = 0, 0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        val_loss = 0  # Optional
        val_precision, val_recall, val_f1, val_rmse, val_roc_auc = calculate_metrics(val_labels, val_preds)

        scheduler.step()

        print(f"Epoch {epoch+1:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    save_path = f"best_resnet18_{'aug' if use_augmented else 'noaug'}.pth"
    torch.save(model.state_dict(), save_path)
    print(f"✅ Best model saved to {save_path}")

    # -------- TESTING --------
    test_loader = DataLoader(test_base, batch_size=batch_size, shuffle=False)
    test_preds, test_labels = [], []

    model.eval()
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_correct = sum(np.array(test_preds) == np.array(test_labels))
    test_acc = test_correct / len(test_labels)
    test_precision, test_recall, test_f1, test_rmse, test_roc_auc = calculate_metrics(test_labels, test_preds)

    # -------- RESULT DICT --------
    result = {
        'Epochs': epochs,
        'Batch Size': batch_size,
        'Learning Rate': learning_rate,
        'Optimizer': 'Adam',
        'Train Accuracy': round(train_acc, 4),
        'Validation Accuracy': round(val_acc, 4),
        'Test Accuracy': round(test_acc, 4),
        'Test Precision': round(test_precision, 4),
        'Test Recall': round(test_recall, 4),
        'Test F1 Score': round(test_f1, 4),
        'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc else None,
        'Test RMSE': round(test_rmse, 4),
        'Augmented': use_augmented
    }

    # -------- SAVE TO CSV --------
    csv_file = 'results.csv'
    write_header = not os.path.exists(csv_file)

    with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=result.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(result)

    print("\n📁 Results saved to results.csv")
    return model

# -------- PREDICT SINGLE IMAGE --------
def predict_skin_type(model_path, image_path):
    model = get_model()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        normalize
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return class_names[predicted.item()]

# -------- MAIN --------
if __name__ == '__main__':
    trained_model = train_model(USE_AUGMENTATION)



========== Training with No Augmentation ==========


d:\sadia\resnet\model\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\sadia\resnet\model\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
d:\sadia\resnet\model\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 01 | Train Acc: 0.7319 | Val Acc: 0.6461


d:\sadia\resnet\model\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 02 | Train Acc: 0.7937 | Val Acc: 0.6397


...

Epoch 09 | Train Acc: 0.9142 | Val Acc: 0.7868


d:\sadia\resnet\model\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 10 | Train Acc: 0.9451 | Val Acc: 0.7974
✅ Best model saved to best_resnet18_noaug.pth


d:\sadia\resnet\model\venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



📁 Results saved to results.csv
